# Check (That Tweet) Yo Self 
## Prioritizing Tweets to Fact Check
###### Part 7: Cluster EDA

Imports

In [1]:
#import modules
import pandas as pd
import numpy as np

#NLP
import os.path
from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Tweets
from tweetscrape.search_tweets import TweetScrapperSearch
from tweetscrape.profile_tweets import TweetScrapperProfile
from tweetscrape.users_scrape import TweetScrapperUser

#Modeling
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#Plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Load in the data:

In [2]:
df = pd.read_csv('../data/user_cluster_tweets.csv')
df.head()

,id,time,author,author_id,associated_tweet,text,links,hashtags,mentions,reply_count,...,ratio,has_url,has_location,has_bio,len_bio,ratio_num_user,emotional_range,user_group_db,user_group,target
0,1254190074595553281,2020-04-25 16:26:30,Iam_helenna,215204985,1254190074595553281,"Today, we have 1182 cases in Nigeria with 35 d...",[],[''],[''],37,...,1.357576,0,0,1,147,0.000000,0.0195,0,0,289
1,1253828209075990531,2020-04-24 16:28:34,KerryeHill,2807727004,1253697753479331840,There's no such thing as a medical disinfectan...,[],[''],[''],1,...,0.241706,0,0,1,89,0.000000,0.0140,0,0,3
2,1253460644294283265,2020-04-23 16:08:00,Lmt48430438,1232381432988930049,1253460644294283265,Waiting to see how many people drink disinfect...,[],[''],['@DarcysCartoon'],1,...,0.357143,0,0,0,3,0.727273,0.0000,1,2,1
3,1254194987945865217,2020-04-25 16:46:01,iamshollyyoung,3096323025,1254194987945865217,Today I know there's no result Nigeria can not...,['https://t.co/RRuHGBH1SI'],['#Covid_19'],[''],1,...,0.470899,1,1,1,103,0.000000,0.0216,2,4,55
4,1253835841685934081,2020-04-24 16:58:54,toddcusuman,588727638,1253835841685934081,New York rapper Fred the Godson dies at 35 aft...,['https://t.co/rXOi5YEoZl'],[''],[''],0,...,0.077656,0,0,0,3,0.000000,0.0000,1,2,0


Taking a look at some of the authors and tweets

In [3]:
df[['author']]

,author
0,Iam_helenna
1,KerryeHill
2,Lmt48430438
3,iamshollyyoung
4,toddcusuman
...,...
33194,adamhume
33195,RebelKanan
33196,CoconutJuice56
33197,JumptyTrumpty


In [4]:
CNN = df[df['author']=='CNN']
CNN

,id,time,author,author_id,associated_tweet,text,links,hashtags,mentions,reply_count,...,ratio,has_url,has_location,has_bio,len_bio,ratio_num_user,emotional_range,user_group_db,user_group,target
688,1253834384232325120,2020-04-24 16:53:06,CNN,759251,1253834384232325120,Vice President Mike Pence says the US has test...,['https://t.co/KgtePUU5fZ'],[''],[''],209,...,43408.86528,1,0,1,154,0.0,0.0000,-1,3,506
2226,1253815503371546625,2020-04-24 15:38:05,CNN,759251,1253815503371546625,Here's a look at who is joining the World Heal...,['https://t.co/euaFe0EQm7'],[''],[''],33,...,43408.86528,1,0,1,154,0.0,0.0000,-1,3,232
17176,1253797389971992577,2020-04-24 14:26:06,CNN,759251,1253797389971992577,State leaders in Indiana are telling residents...,['https://t.co/ZOJGovOh2H'],[''],[''],169,...,43408.86528,1,0,1,154,0.0,0.0000,-1,3,829
18215,1254128567178452994,2020-04-25 12:22:05,CNN,759251,1254128567178452994,As people look for an outlet to exercise and e...,['https://t.co/Tj56cxUwB5'],[''],[''],66,...,43408.86528,1,0,1,154,0.0,0.0078,-1,3,751
21057,1253824051153428481,2020-04-24 16:12:03,CNN,759251,1253824051153428481,The company that makes Lysol is urging custome...,['https://t.co/XMPlLKqxJX'],[''],[''],112,...,43408.86528,1,0,1,154,0.0,0.0085,-1,3,861
23502,1253827243765366784,2020-04-24 16:24:44,CNN,759251,1253827243765366784,The Supreme Court denied a request to block th...,['https://t.co/Fu9b3pA6J7'],[''],[''],59,...,43408.86528,1,0,1,154,0.0,0.0392,-1,3,344
24046,1253831853955571712,2020-04-24 16:43:03,CNN,759251,1253831853955571712,The 2020 California State Fair and Food Festiv...,['https://t.co/98Soznhpez'],[''],[''],21,...,43408.86528,1,0,1,154,0.0,0.0268,-1,3,259
28388,1253824051228954628,2020-04-24 16:12:03,CNN,759251,1253824051228954628,The economic fallout of the coronavirus pandem...,['https://t.co/LgOYv4V0Tr'],[''],[''],12,...,43408.86528,1,0,1,154,0.0,0.0078,-1,3,133


In [5]:
CNN.text[688]

'Vice President Mike Pence says the US has tested roughly 5.1 million Americans for coronavirus https://cnn.it/2S6FMX9'

In [6]:
CNN.text[2226]

"Here's a look at who is joining the World Health Organization's new vaccine efforts. The WHO announced the launch of a new effort to accelerate the development of vaccines, diagnostics and therapeutics in response to the coronavirus pandemic. https://cnn.it/2S5eDnA"

### DBSCAN Clusters
The section below explores how the clusters differ from each other.

In [7]:
df['user_group_db'].value_counts(normalize = True)

 0    0.494533
 3    0.221151
 1    0.127926
-1    0.057321
 4    0.053255
 2    0.045815
Name: user_group_db, dtype: float64

About half of the observations fall into group 0 and almost 6% are outliers.

In [8]:
df.groupby(by='user_group_db')['target'].mean()

user_group_db
-1    148.223857
 0     21.817944
 1      2.706146
 2      8.134122
 3     30.803187
 4     10.931561
Name: target, dtype: float64

There's a large different in target between the groups, even though this was not considered when clustering.

In [9]:
#DBSCAN Clusters
df.groupby(by='user_group_db')[['target', 'retweet_count', 'favorite_count', 'reply_count', 'link_count',  
                               'ratio', 'text_sentiment', 'emotional_range']].mean()

,target,retweet_count,favorite_count,reply_count,link_count,ratio,text_sentiment,emotional_range
user_group_db,,,,,,,,
-1,148.223857,27.337888,114.163952,6.722018,0.479243,723.734861,-0.057507,0.018640
0,21.817944,3.998599,16.968023,0.851322,0.259471,1.621993,-0.044526,0.019169
1,2.706146,0.273605,2.052272,0.380268,0.214269,1.127193,-0.048747,0.019135
2,8.134122,1.370809,5.859303,0.904011,0.362262,2.813750,-0.014522,0.018795
3,30.803187,5.484609,23.609235,1.709344,0.429583,11.895564,0.018602,0.019445
4,10.931561,2.361425,7.983032,0.587104,0.263009,1.995957,-0.056170,0.019189


Above we can see some of the areas the groups differ most by looking at the average. Target and ratio have a lot of variance while emotional range is more consistent between the groups.

### Top Retweets and Retweet Count

In [10]:
# Highest Retweet_Count
df[df['user_group_db'] != 5][['author', 'user_followers', 'user_following', 'user_tweets', 'retweet_count',
                               'ratio', 'user_group_db']].sort_values('retweet_count', ascending=False)[:25]

,author,user_followers,user_following,user_tweets,retweet_count,ratio,user_group_db
29909,EM_RESUS,81269,160,7015,28381,507.931250,0
27908,MollyJongFast,517766,5376,193092,15323,96.310640,-1
19287,signofline,2532,408,13028,15305,6.205882,0
1929,MarkDice,608614,332,5044,11091,1833.174699,-1
17726,AnnaESanders,8573,999,14959,6402,8.581582,3
23699,waltshaub,502382,1298,5686,5371,387.043143,0
203,HKrassenstein,47056,4662,3418,3340,10.093522,4
15120,cjtruth,128613,21233,115391,2941,6.057222,-1
21386,EmmaWatson,29161374,394,1793,2568,74013.639594,-1
14559,DeAnna4Congress,234459,70290,14610,2440,3.335595,-1


#### Adding Context: Twitter Accounts with the highest retweet count
    
**EM_RESUS** - Sam Ghali, M.D. @EM_RESUS USA - ER Doctor. Patient Advocate. <br>
**MollyJongFast** - Molly Jong-Fast, @MollyJongFast. Editor at large @thedailybeast.<br>
**signofline** - Individual<br>
**MarkDice** - Mark Dice, Media Analyst. Author of "The Liberal Media Industrial Complex."<br>
**AnnaESanders** - City Hall reporter for the New York Daily News. NY, NY.<br>
**waltshaub** - Walter Shaub @waltshaub, former Director of @OfficeGovEthics personal account - views my own Tweets are *not* legal advice.<br>
**HKrassenstein** - Individual<br>
**cjtruth** - Individual, active user<br>
**EmmaWatson** - Public Figure. Actor.<br>
**DeAnna4Congress** - Fmr Congressional Candidate vs Nancy Pelosi (Republican/Author #MAGA➡️Join me in the fight!)<br>
**BrianKarem** - White House reporter.<br>
**NatashaBertrand** - Natasha Bertrand. National security correspondent @politico. @MSNBC contributor. Washington, DC.<br>

**Observations**: the most re-tweeted tweets originated from a variety of public and private individual accounts, in oppose to a traditional media source like CNN, BBC, or AP. We may infer that tweets generated by individuals are more likely to be re-tweeted, and likely have higher trustworthiness and higher engagement within the active twitter community. 

In [11]:
df['retweet_count'].sort_values(ascending=False)

29909    28381
27908    15323
19287    15305
1929     11091
17726     6402
         ...  
21127        0
21128        0
21129        0
21130        0
16599        0
Name: retweet_count, Length: 33199, dtype: int64

Above we can see the full range of retweets in our dataset.

In [12]:
#most retweets
df.sort_values(by=['retweet_count'], ascending=False)[:5]

,id,time,author,author_id,associated_tweet,text,links,hashtags,mentions,reply_count,...,ratio,has_url,has_location,has_bio,len_bio,ratio_num_user,emotional_range,user_group_db,user_group,target
29909,1253458355517042689,2020-04-23 15:58:54,EM_RESUS,2741335327,1253458355517042689,"Hi, ER Doc here.Do NOT inject or consume ANY d...",[],[''],[''],2778,...,507.931250,0,0,1,148,0.0,0.0239,0,0,143818
27908,1253465460005392384,2020-04-23 16:27:08,MollyJongFast,14298769,1253465460005392384,We now have to tell people not to ingest Cloro...,[],[''],[''],2904,...,96.310640,1,0,1,86,0.0,0.0180,-1,5,104596
19287,1254196836471955456,2020-04-25 16:53:22,signofline,1025955661819998208,1254196836471955456,girls don’t want boyfriends they want the coro...,[],[''],[''],286,...,6.205882,0,0,1,17,0.0,0.0254,0,0,77188
1929,1254182411492929536,2020-04-25 15:56:03,MarkDice,35039490,1254182411492929536,Trump used the word “disinfectant” meaning med...,[],[''],[''],1545,...,1833.174699,1,0,1,140,0.0,0.0253,-1,1,48397
17726,1253818333243355136,2020-04-24 15:49:20,AnnaESanders,134622111,1253818333243355136,NYC Poison Control Center saw 30 cases of expo...,['https://t.co/fVAUvGmuwM'],[''],[''],868,...,8.581582,1,0,1,155,0.0,0.0174,3,1,15948


In [13]:
df.groupby(by='user_group_db')['emotional_range'].mean()

user_group_db
-1    0.018640
 0    0.019169
 1    0.019135
 2    0.018795
 3    0.019445
 4    0.019189
Name: emotional_range, dtype: float64

In [14]:
df['retweet_count'].mean()

4.9809632820265675

The average number of retweets for all the tweets is just under 5.

### Cluster 3
Besides our outliers, cluster 3 seems to have the biggest potential reach so we'll explore that cluster further below.

In [15]:
df[df['user_group_db'] == 3].describe()

,id,author_id,associated_tweet,reply_count,favorite_count,retweet_count,not_english,hashtag_count,mention_count,word_count,...,ratio,has_url,has_location,has_bio,len_bio,ratio_num_user,emotional_range,user_group_db,user_group,target
count,7.342000e+03,7.342000e+03,7.342000e+03,7342.000000,7342.000000,7342.000000,7342.000000,7342.000000,7342.000000,7342.000000,...,7342.000000,7342.0,7342.0,7342.0,7342.000000,7342.000000,7342.000000,7342.0,7342.0,7342.000000
mean,1.253964e+18,2.590995e+17,1.253206e+18,1.709344,23.609235,5.484609,0.284257,0.422501,0.177200,25.865568,...,11.895564,1.0,0.0,1.0,105.836012,0.033074,0.019445,3.0,1.0,30.803187
std,2.457362e+14,4.500994e+17,1.742627e+16,17.998160,332.603277,96.377925,0.101317,1.195927,0.574448,13.224304,...,75.811201,0.0,0.0,0.0,47.000649,0.093433,0.014145,0.0,0.0,416.955815
min,1.253229e+18,2.656000e+03,4.495253e+17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.004335,1.0,0.0,1.0,1.000000,0.000000,0.000000,3.0,1.0,0.000000
25%,1.253825e+18,7.163071e+07,1.253812e+18,0.000000,0.000000,0.000000,0.214300,0.000000,0.000000,15.000000,...,0.570954,1.0,0.0,1.0,68.000000,0.000000,0.009200,3.0,1.0,0.000000
50%,1.254110e+18,4.940871e+08,1.253835e+18,0.000000,1.000000,0.000000,0.285700,0.000000,0.000000,24.000000,...,1.059919,1.0,0.0,1.0,118.000000,0.000000,0.018700,3.0,1.0,1.000000
75%,1.254190e+18,7.204643e+17,1.254186e+18,1.000000,3.000000,0.000000,0.359000,0.000000,0.000000,37.000000,...,2.482964,1.0,0.0,1.0,149.000000,0.000000,0.028400,3.0,1.0,5.000000
max,1.254199e+18,1.253859e+18,1.254199e+18,868.000000,20044.000000,6402.000000,0.491500,14.000000,11.000000,60.000000,...,1907.500000,1.0,0.0,1.0,183.000000,0.875000,0.100000,3.0,1.0,21104.000000


In [16]:
# most retweets
df[df['user_group_db'] == 3][['author', 'user_followers', 'user_following', 'user_tweets', 
                              'retweet_count','ratio']].sort_values('retweet_count', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,retweet_count,ratio
17726,AnnaESanders,8573,999,14959,6402,8.581582
28001,BrianKarem,277139,1371,15843,2197,202.143691
22666,NatashaBertrand,630216,5201,23596,2056,121.172082
11254,ECMcLaughlin,97079,1500,35113,1806,64.719333
3966,Politidope,43273,14459,134485,1733,2.992807
3065,morethanmySLE,92468,13663,89341,1295,6.767767
5566,PadmaLakshmi,662787,1556,11801,1211,425.955656
27087,DavaStarr,5292,3079,123133,1129,1.718740
20906,ASlavitt,439849,976,31647,981,450.664959
17399,bocxtop,87265,698,9494,979,125.021490


In [17]:
#most tweets
df[df['user_group_db'] == 3][['author', 'user_followers', 'user_following', 
                              'user_tweets', 'retweet_count','ratio']].sort_values('user_tweets', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,retweet_count,ratio
9668,_heymynameisNAY,458,207,166071,1,2.212560
19247,AyyAmbs,5501,657,165456,1,8.372907
7549,bluebay700,1375,509,165362,0,2.701375
17908,Mario__Bones,1227,2046,165221,0,0.599707
1307,perfectrose2011,1664,1489,165151,0,1.117529
18802,perfectrose2011,1664,1489,165151,0,1.117529
5877,AdaptToReality,2283,2563,165002,0,0.890753
24068,RocketInTheMix,5506,4533,164504,0,1.214648
31837,MassiveDumps4U,1710,589,164343,0,2.903226
12561,insert_funny,1848,3234,164227,0,0.571429


In [18]:
#most followers
df[df['user_group_db'] == 3][['author', 'user_followers', 'user_following', 
                              'user_tweets', 'retweet_count','ratio', 'time']].sort_values('user_followers', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,retweet_count,ratio,time
8926,LAist,786032,3091,93550,9,254.296991,2020-04-25 16:47:01
1331,ForeignAffairs,748417,2003,64054,579,373.648028,2020-04-25 16:33:32
3435,TeamCanada,731390,1941,37234,30,376.810922,2020-04-25 16:43:55
21147,sjokz,688395,1271,54908,51,541.616837,2020-04-25 16:37:35
13002,FreeformTV,687465,853,94526,8,805.937866,2020-04-25 12:00:00
5566,PadmaLakshmi,662787,1556,11801,1211,425.955656,2020-04-23 15:55:46
22434,500Startups,648281,2990,22393,9,216.816388,2020-04-24 16:14:03
2907,danpfeiffer,634546,3534,28577,71,179.554612,2020-04-23 16:11:58
17582,AprilDRyan,630616,5196,56985,39,121.365666,2020-04-24 15:55:04
22666,NatashaBertrand,630216,5201,23596,2056,121.172082,2020-04-24 16:01:43


In [19]:
#DBSCAN Clusters
df.groupby(by='user_group_db')[['target', 'retweet_count', 'favorite_count', 'reply_count', 'link_count',  
                               'ratio', 'text_sentiment', 'emotional_range']].mean()

,target,retweet_count,favorite_count,reply_count,link_count,ratio,text_sentiment,emotional_range
user_group_db,,,,,,,,
-1,148.223857,27.337888,114.163952,6.722018,0.479243,723.734861,-0.057507,0.018640
0,21.817944,3.998599,16.968023,0.851322,0.259471,1.621993,-0.044526,0.019169
1,2.706146,0.273605,2.052272,0.380268,0.214269,1.127193,-0.048747,0.019135
2,8.134122,1.370809,5.859303,0.904011,0.362262,2.813750,-0.014522,0.018795
3,30.803187,5.484609,23.609235,1.709344,0.429583,11.895564,0.018602,0.019445
4,10.931561,2.361425,7.983032,0.587104,0.263009,1.995957,-0.056170,0.019189


### Types of sources within DBSCAN Cluster 3

**Observations**: The Cluster 3 has the second highest target score among all clusters, which indicates the second highest potential for tweets to have a higher reach or get retweeted. Interestingly, this is also the only cluster with the positive mean sentiment.


**Top Tweets by Number of Followers (user_followers):**<br>
1. LAist<br>
2. ForeignAffairs<br>
3. TeamCanada - Official Twitter Account of the Canadian Opympic Team<br>
4. sjokz - Eefje Depoortere, freelance esports host (Berlin, Germany)<br>
5. FreeformTV - a new (4-yrs old) channel. Freeform is an American basic cable channel owned by Walt Disney Television, a unit of Disney Media Networks and a part of The Walt Disney Company<br>
6. PadmaLakshmi - Host of Bravo Top Chef, NYC<br>
7. 500Startups - 500 Startups is an early-stage venture fund and seed accelerator founded in 2010 by Dave McClure and Christine Tsai. Mountain View, CA. Funded many companies, among which are Udemy, GitLab, Tradesy, Winc.<br>
8. danpfeiffer - Howard Daniel Pfeiffer is an American activist, podcaster, and former Senior Advisor to U.S. President Barack Obama for Strategy and Communications<br>
9. AprilDRyan - CNN Political analyst, White House Correspondent & DC Bureau Chief for @AURNonline<br>
10. NatashaBertrand - Natasha Bertrand is an American journalist who is a national security correspondent for Politico. She is also a political analyst for NBC and MSNBC<br>


**Top Tweets by Retweet_count:**<br>
1. AnnaESanders - New York, New York, City Hall reporter for the New York Daily News.<br>
2. BrianKarem - WH Reporter, Washington DC<br>
3. ECMcLaughlin - CEO, Gaia Project for Women's Leadership<br>
4. Politidope - Matt Rogers, Chief Strategist at Politidope, Inc.<br>
5. morethanmySLE - Peter Morley, Patient & Patient Advocate Testified @AppropsDems/@EnergyCommerce/@OversightDems<br>
6. PadmaLakshmi - Host of Bravo Top Chef, NYC<br>
7. DavaStarr - Individual<br>
8. ASlavitt - Andy Slavitt, Ex-Obama health care head. Helper. Founded @usofcare & @townhallvntrs.<br>
9. bocxtop - Individual<br>


**Top Tweets by Number of Tweets (user_tweets):**<br>
1. _heymynameisNAY - individual, Brooklyn, NY <br>
2. AyyAmbs - individual<br>
3. bluebay700 - Lifestyle magazine and online publishing, LGBT community<br>
4. Mario__Bones - individual, Australia<br>
5. perfectrose2011 - individual, Canada<br>
6. AdaptToReality - individual<br>
7. RocketInTheMix - individual, dj<br>
8. MassiveDumps4U - individual<br>
9. insert_funny - individual<br>


# Other Clusters
We'll also take a look at the other clusters to explore what characteristics make it unique and what users fall into each.

### DBSCAN Cluster -1

In [20]:
#DBSCAN Clusters
df.groupby(by='user_group_db')[['target', 'retweet_count', 'favorite_count', 'reply_count', 'link_count',  
                               'ratio', 'text_sentiment', 'emotional_range']].mean()

,target,retweet_count,favorite_count,reply_count,link_count,ratio,text_sentiment,emotional_range
user_group_db,,,,,,,,
-1,148.223857,27.337888,114.163952,6.722018,0.479243,723.734861,-0.057507,0.018640
0,21.817944,3.998599,16.968023,0.851322,0.259471,1.621993,-0.044526,0.019169
1,2.706146,0.273605,2.052272,0.380268,0.214269,1.127193,-0.048747,0.019135
2,8.134122,1.370809,5.859303,0.904011,0.362262,2.813750,-0.014522,0.018795
3,30.803187,5.484609,23.609235,1.709344,0.429583,11.895564,0.018602,0.019445
4,10.931561,2.361425,7.983032,0.587104,0.263009,1.995957,-0.056170,0.019189


Total averages for comparison:

In [21]:
df['retweet_count'].mean()

4.9809632820265675

In [22]:
df[['target','retweet_count', 'favorite_count', 'reply_count']].mean()

target            27.399199
retweet_count      4.980963
favorite_count    21.112564
reply_count        1.305672
dtype: float64

### Types of sources within Outliers

**Observations**: The outliers have the highest target score among all groupings, which indicates the highest potential for tweets to be impactful. Based on our EDA, these outliers are marked by high follower count and thus high impact capability. Many of these tweets are from public figures or traditional news sources like CNN, WSJ, AP, USAToday etc.  

In terms of our problems statement, we feel like it is appropriate to consider these outliers. The purpose of this model is to identify tweets that should be fact checked. Solely fact checking sources that are often regulated or have such a large following, trying to correct them would not have any impact. Instead, we should focus on tweets where course correcting the user is possible.

In [23]:
#most followers
df[df['user_group_db'] == -1][['author', 'user_followers', 'user_following', 
                              'user_tweets', 'retweet_count','ratio', 'time']].sort_values('user_followers', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,retweet_count,ratio,time
21057,CNN,48010205,1106,300064,182,43408.865280,2020-04-24 16:12:03
28388,CNN,48010205,1106,300064,35,43408.865280,2020-04-24 16:12:03
23502,CNN,48010205,1106,300064,81,43408.865280,2020-04-24 16:24:44
18215,CNN,48010205,1106,300064,110,43408.865280,2020-04-25 12:22:05
2226,CNN,48010205,1106,300064,41,43408.865280,2020-04-24 15:38:05
688,CNN,48010205,1106,300064,56,43408.865280,2020-04-24 16:53:06
24046,CNN,48010205,1106,300064,52,43408.865280,2020-04-24 16:43:03
17176,CNN,48010205,1106,300064,149,43408.865280,2020-04-24 14:26:06
21386,EmmaWatson,29161374,394,1793,2568,74013.639594,2020-04-24 14:57:11
25907,WSJ,17745806,917,307055,40,19352.023991,2020-04-25 13:15:00


In [24]:
#most tweets
df[df['user_group_db'] == -1][['author', 'user_followers', 'user_following', 
                              'user_tweets', 'retweet_count','ratio']].sort_values('user_tweets', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,retweet_count,ratio
10684,PulpNews,4752,941,3430906,0,5.049947
29807,Airtel_Presence,418392,80517,2782254,0,5.196319
31025,TomthunkitsMind,70832,67642,2423468,2,1.047160
28222,TomthunkitsMind,70832,67642,2423468,2,1.047160
26059,TomthunkitsMind,70832,67642,2423468,5,1.047160
4701,filafresh,7413,4219,1922381,0,1.757051
30126,Uskeus,4905,224,1854890,0,21.897321
33092,Chris_1791,76750,43768,1458147,0,1.753564
44,commonpatriot,15483,13960,1215321,0,1.109097
16942,JoeFreedomLove,67829,68315,1089226,2,0.992886


The DataFrames below take a closer look at the accounts in each cluster.

In [25]:
df[df['user_group_db'] == 0][['author', 'user_followers', 'user_following', 'user_tweets', 'ratio']].sort_values('user_followers', ascending=False)[:30]

,author,user_followers,user_following,user_tweets,ratio
23699,waltshaub,502382,1298,5686,387.043143
19299,waltshaub,502382,1298,5686,387.043143
3050,BrookeBCNN,316570,2310,30323,137.043290
13565,Khanoisseur,145287,9842,57736,14.761939
2259,Khanoisseur,145287,9842,57736,14.761939
24461,4everNeverTrump,136539,1970,95863,69.309137
18985,ToryFibs,132664,19697,67068,6.735239
4462,page88,130268,3859,42228,33.756932
28023,KFANRosen,121595,4862,29806,25.009255
10740,ChadPergram,118448,18298,101112,6.473276


In [26]:
df[df['user_group'] == 5][['author', 'user_tweets', 'user_followers', 'ratio']].sort_values('user_tweets', ascending=False)[:30]

,author,user_tweets,user_followers,ratio
44,commonpatriot,1215321,15483,1.109097
19935,JoeFreedomLove,1089226,67829,0.992886
16942,JoeFreedomLove,1089226,67829,0.992886
18981,Ez4u2say_Janis,1047654,12146,0.977073
1657,WLOYisPlaying,1046394,1649,1649.000000
12604,threadreaderapp,1022473,321888,248.370370
28898,threadreaderapp,1022473,321888,248.370370
7589,akemoi,995363,2142,1.826087
12163,Mr1738,986032,14775,36.571782
11696,cherokeesher2,966804,33814,1.163753


Additionally, we spent a lot of time actually going to these user's profiles to fully understand the groupings.